https://arxiv.org/abs/1711.05772 / https://arxiv.org/abs/1802.04877

https://github.com/natashamjaques/magenta/blob/affective-reward/magenta/models/affective_reward/latent_gan.py

In [1]:
import torch

print('cuda.is_available:', torch.cuda.is_available())
print(f'available: {torch.cuda.device_count()}; current: {torch.cuda.current_device()}')
DEVICE = torch.device(f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
print('pytorch', torch.__version__)

cuda.is_available: True
available: 1; current: 0
cuda:0
pytorch 0.4.0


## Labeling data

In [2]:
#from pdb import set_trace
#set_trace()

In [3]:
import numpy as np

labels = np.empty([0, 2], dtype=int)
gens_keep = np.empty([0, ], dtype=object)
zs_keep = np.empty([0, 1, 128], dtype=float)

In [4]:
# pip install git+https://github.com/iconix/pytorch-text-vae.git
from pytorchtextvae import generate

In [5]:
DEVICE = torch.device('cpu') # CPU inference
n_samples = 100
temp = 0.2

# workaround for un-pickling after module directory change https://stackoverflow.com/a/45264751
#import sys
#sys.path.append('../../pytorch-text-vae/pytorchtextvae')

vae, input_side, output_side, pairs, dataset, EMBED_SIZE, random_state = generate.load_model('../../pytorch-text-vae/model/best/reviews_and_metadata_5yrs_state.pt', 'reviews_and_metadata_5yrs_stored_info.pkl', DEVICE, cache_path='../../pytorch-text-vae/model/best/tmp')

Fetching cached info at ../../pytorch-text-vae/model/best/tmp/reviews_and_metadata_5yrs_stored_info.pkl
Cache ../../pytorch-text-vae/model/best/tmp/reviews_and_metadata_5yrs_stored_info.pkl loaded (load time: 0.80s)
Found saved model ../../pytorch-text-vae/model/best/reviews_and_metadata_5yrs_state.pt
MAX_SAMPLE: False; TRUNCATED_SAMPLE: True
Trained for 360000 steps (load time: 19.40s)
Setting new random seed


In [6]:
generate.generate(vae, input_side, output_side, pairs, dataset, EMBED_SIZE, random_state, DEVICE, genres=['downtempo', 'dream pop', 'indietronica'], num_sample=10, temp=temp)

---
['downtempo', 'dream pop', 'indietronica']
electronic musician artist has just dropped a new album and UNK is the second time for the UNK week
---
['downtempo', 'dream pop', 'indietronica']
i was like a much of late UNK UNK but also built on more tour
---
['downtempo', 'dream pop', 'indietronica']
if his debut full length album was a big fan of the album set to be produced by UNK UNK and brooklyn based based electric UNK UNK UNK in electric sounds
---
['downtempo', 'dream pop', 'indietronica']
two UNK two tracks that is taken that that come to it it with a later it is no spring
---
['downtempo', 'dream pop', 'indietronica']
this is a unique sound that is a catchy pop groove to get into a hip hop beat and again for the top of their career
---
['downtempo', 'dream pop', 'indietronica']
this is UNK UNK UNK UNK UNK UNK and artist return with UNK UNK and catch artist live
---
['downtempo', 'dream pop', 'indietronica']
its the kind of UNK UNK UNK UNK UNK to the UNK with a a UNK to be som

(['electronic musician artist has just dropped a new album and UNK is the second time for the UNK week',
  'i was like a much of late UNK UNK but also built on more tour',
  'if his debut full length album was a big fan of the album set to be produced by UNK UNK and brooklyn based based electric UNK UNK UNK in electric sounds',
  'two UNK two tracks that is taken that that come to it it with a later it is no spring',
  'this is a unique sound that is a catchy pop groove to get into a hip hop beat and again for the top of their career',
  'this is UNK UNK UNK UNK UNK UNK and artist return with UNK UNK and catch artist live',
  'its the kind of UNK UNK UNK UNK UNK to the UNK with a a UNK to be something that nature',
  'its sort of UNK UNK before its also the second taste of his album before throughout july',
  'UNK UNK UNK UNK of the year and they deliver another different rest of this full length',
  'in and is like to to to different and but with the the different sounds like their in

In [7]:
gens, zs, conditions = generate.generate(vae, input_side, output_side, pairs, dataset, EMBED_SIZE, random_state, DEVICE, num_sample=n_samples, temp=temp)

---
['deep tropical house', 'edm', 'indie poptimism', 'tropical house', 'vapor soul', 'vapor twitch']
what happens finally get you with the some of being featured on my night as you were all over right
---
['alternative rock', 'art pop', 'chamber pop', 'dream pop', 'folk-pop', 'freak folk', 'indie folk', 'indie pop', 'indie rock', 'indietronica', 'lo-fi', 'modern rock', 'neo-psychedelic', 'noise pop', 'soul', 'stomp and holler']
UNK UNK better makes for UNK UNK UNK UNK UNK UNK UNK UNK better UNK UNK UNK UNK voice of UNK to break
---
['chamber pop', 'electropop', 'indie r&b', 'indietronica', 'vapor soul']
her first tour and sound that is right that get right in spring right in its lead
---
['indie poptimism']
at the we are decided to come out with the such as we didnt think within the wait
---
['alternative dance', 'balearic', 'dance-punk', 'electronic', 'microhouse', 'new rave', 'nu disco']
the image courtesy of artist will be now via dance music artist or something fine
---
['brooklyn

---
['hip hop', 'pop', 'pop rap', 'rap', 'southern hip hop', 'trap music']
song makes for a sense of hip hop and but in the kind of hip world but that is all all you going to familiar with hip hop
---
['deep tropical house', 'indie poptimism', 'tropical house', 'vapor soul']
music has teamed up with free download of this track which has it up for free days
---
['gauze pop', 'pop', 'tropical house']
at the beginning a UNK self UNK is out in a song and UNK has to has another amazing here
---
['aussietronica', 'deep australian indie', 'indie electro-pop', 'vapor soul']
recently signed UNK UNK hailing from UNK because of a nice to share with nice writing
---
['big room', 'edm', 'electro house', 'progressive house']
caught up with his his UNK UNK is exactly what its going to pick up a work in UNK and its just one two tracks under this UNK
---
['future garage', 'indie r&b', 'vapor soul']
oh given that this sultry vocals and there is a perfect addition to the chilled out vocals shine
---
['ga

In [8]:
list(zip(range(len(gens)), gens))

[(0,
  'what happens finally get you with the some of being featured on my night as you were all over right'),
 (1,
  'UNK UNK better makes for UNK UNK UNK UNK UNK UNK UNK UNK better UNK UNK UNK UNK voice of UNK to break'),
 (2,
  'her first tour and sound that is right that get right in spring right in its lead'),
 (3,
  'at the we are decided to come out with the such as we didnt think within the wait'),
 (4,
  'the image courtesy of artist will be now via dance music artist or something fine'),
 (5,
  'but as but artist cut from his forthcoming debut album that is just as just as its about we love before'),
 (6,
  'having started together on the remix of UNK and have teamed up with a stellar track for me'),
 (7,
  'a lot of the project that reminds recorded in a couple of life and we grew up up a social club fact that we recorded it later this summer'),
 (8,
  'get better with their latest remix of brooklyn based trio artist has teamed up with the press beat at just one mark'),
 (9,

In [9]:
def to_embed(z, condition):
    if condition.dim() == 1:
        condition = condition.unsqueeze(0)
    squashed_condition = vae.decoder.c2h(condition)
    return torch.cat([z, squashed_condition], 1)

In [10]:
n_latent = 128
from pytorchtextvae.datasets import EOS_token

def generate(condition, gan=None, z=None, max_sample=False, truncated_sample=True, temp=temp):
    with torch.no_grad():
        if gan is None:
            z_prime = z
        else:
            gan.eval()
            z = torch.randn(1, n_latent).to(DEVICE)
            decode_embed = to_embed(z, condition).to(DEVICE)
            z_prime = gan.G(decode_embed)

        generated = vae.decoder.generate_with_embed(z_prime, 50, temp, DEVICE, max_sample=max_sample, trunc_sample=truncated_sample)
        generated_str = model.float_word_tensor_to_string(output_side, generated)

        EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

        if generated_str.endswith(EOS_str):
            generated_str = generated_str[:-5]

        # flip it back
        return generated_str[::-1], z, z_prime

### 'Banned' approach

label a sample as -1 (=="bad") if it contains a banned word; label as 1 otherwise (=="good")

In [7]:
#new_labels = np.array([(1, -1), (10, -1)])

banned = ['below']
labels = np.ones(n_samples, dtype=int)
gens_lose = list(set([i for b in banned for i in np.where([b in g.split() for g in gens])[0]]))
labels[gens_lose] = -1
zs_keep = zs

labels

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1])

In [8]:
from collections import Counter
c1 = Counter([word for gen in gens for word in gen.split()])
[(b, c1[b]) for b in banned], c1.most_common(25)

([('below', 3)],
 [('the', 94),
  ('UNK', 82),
  ('of', 67),
  ('a', 66),
  ('and', 59),
  ('to', 55),
  ('is', 29),
  ('as', 26),
  ('artist', 26),
  ('with', 26),
  ('from', 24),
  ('it', 21),
  ('this', 21),
  ('track', 20),
  ('on', 20),
  ('up', 19),
  ('one', 18),
  ('be', 15),
  ('in', 14),
  ('has', 14),
  ('at', 11),
  ('i', 10),
  ('been', 10),
  ('trying', 10),
  ('that', 10)])

### 'Realism' approach

label a sample as 1 (=="good") if it came from the training data; label as -1 (=="bad") if it came from a random Gaussian `z`

In [10]:
from pytorchtextvae import model

input, target, condition = model.random_training_set(pairs, input_side, output_side, random_state, DEVICE)
model.long_word_tensor_to_string(input_side, input), dataset.decode_genres(condition)

('newcomer artist released his debut single last week and its already gaining major attention and a following that is demanding more after fill EOS ',
 ['vapor soul'])

In [11]:
temperature = 1.0

m, l, z, decoded = vae(input, target, condition, DEVICE, temperature)

z.size(), decoded.size()

(torch.Size([1, 128]), torch.Size([24, 333336]))

In [12]:
generate(condition, z=z, max_sample=True)[0]

'artist released his debut single and is just released and more than a ago and that that get your attention'

**TODO:** shouldn't generate with max sampling always return the same sample?

Even though the encoding is imperfect, we will still consider these `z`s as "realistic"

In [13]:
real_z = []
real_gens = []
for i in range(int(n_samples/2)):
    input, target, condition = model.random_training_set(pairs, input_side, output_side, random_state, DEVICE)
    with torch.no_grad():
        _, _, z, _ = vae(input, target, condition, DEVICE, temperature)
        real_z.append(z)
        real_gens.append(generate(condition, z=z, max_sample=True)[0])

In [14]:
from collections import Counter
c1 = Counter([word for gen in real_gens for word in gen.split()])
c1.most_common(25)

[('UNK', 106),
 ('and', 53),
 ('the', 50),
 ('a', 30),
 ('of', 30),
 ('on', 28),
 ('to', 24),
 ('in', 15),
 ('is', 15),
 ('i', 14),
 ('with', 14),
 ('w', 13),
 ('even', 11),
 ('that', 10),
 ('for', 10),
 ('who', 10),
 ('this', 9),
 ('always', 9),
 ('it', 9),
 ('tour', 8),
 ('into', 8),
 ('be', 8),
 ('but', 8),
 ('you', 7),
 ('new', 7)]

In [15]:
# up until now, `zs` held random zs - now concat with real zs
zs = torch.cat((torch.stack(real_z).squeeze(), torch.stack(zs[:int(n_samples/2)]).squeeze()))

In [16]:
labels = np.ones(n_samples, dtype=int)
labels[range(len(real_z), len(zs))] = -1
embeds = zs

labels

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

### Realism + Readability
in addition to the realism discriminator, add readability as a conditioning attribute

In [14]:
# https://github.com/shivam5992/textstat/issues/43
from textstat.textstat import textstat

# Score 	 Difficulty
# 90-100 	 Very Easy
# 80-89 	 Easy
# 70-79 	 Fairly Easy
# 60-69 	 Standard
# 50-59 	 Fairly Difficult
# 30-49 	 Difficult
# 0-29 	 Very Confusing

[textstat.flesch_reading_ease(sent) for sent in ["This is a sentence", "To be or not to be", ]]

[92.8, 116.15]

In [26]:
[(' '.join(gen.replace('UNK', '').split()), textstat.text_standard(' '.join(gen.replace('UNK', '').split()))) for gen in np.array(real_gens)[random_state.choice(len(real_gens), 10)]]

[('one of those who dont want to the one of their own who are their own in their own music',
  '4th and 5th grade'),
 ('the remix the production from the original and it sounds like a it with the bass and that sounds like it sounds like it from the chorus',
  '6th and 7th grade'),
 ('we been waiting for the last year while while its soon as waiting for little while we we as we as it',
  '12th and 13th grade'),
 ('one of those who dont want to the one of their own who are their own in their own music',
  '4th and 5th grade'),
 ('we trying to get into an artist is at the one of the and in the song that a part of the is in the trying to into it into a song to artist is just into part into that that or else',
  '14th and 15th grade'),
 ('if youre on the first version of you of know if you hear the version of this is that is to a song',
  '1th and 2th grade'),
 ('we been waiting for the last year while while its soon as waiting for little while we we as we as it',
  '12th and 13th grade'),


### Topical
Prefer certain topics to others

In [11]:
import pytorchtextvae.datasets as ds

def tokenize(line):
    l = line.strip().lstrip().rstrip()
    l = ds.normalize_string(l)
    return l.split(' ')

In [12]:
n_examples = 3

sents = [pair[0] for pair in pairs]
texts = [tokenize(sentence) for sentence in sents]
texts[:n_examples]

[['artist',
  'just',
  'announced',
  'their',
  'biggest',
  'headlining',
  'show',
  'to',
  'date',
  'a',
  'december',
  '13th',
  'concert',
  'at',
  'the',
  '20',
  '000',
  'capacity',
  'london',
  'o2',
  'arena'],
 ['and',
  'then',
  'there',
  'are',
  'the',
  'people',
  'youd',
  'never',
  'talk',
  'to',
  'if',
  'they',
  'werent',
  'handing',
  'you',
  'a',
  'paycheck'],
 ['posted',
  'on',
  'june',
  '20',
  '2014',
  'in',
  'stream',
  'and',
  'tagged',
  'artist',
  'outsiders',
  'polydor'],
 ['cop',
  'a',
  'free',
  'download',
  'of',
  'that',
  'one',
  'via',
  'bearson',
  's',
  'soundcloud',
  'page'],
 ['i',
  've',
  'developed',
  'this',
  'protective',
  'feeling',
  'about',
  'how',
  'were',
  'portrayed',
  'and',
  'at',
  'the',
  'same',
  'time',
  'im',
  'acutely',
  'aware',
  'of',
  'the',
  'issues',
  'we',
  'face',
  'and',
  'the',
  'root',
  'causes',
  'of',
  'these',
  'issues']]

In [13]:
from nltk.corpus import stopwords

# remove stop words and words that appear only once
stoplist = [ds.normalize_string(word) for word in stopwords.words('english')]
fillerlist = ['author', 'song_title', 'artist', 'sitename']

texts = [[word for word in text if word not in stoplist and word not in fillerlist] for text in texts]
texts[:n_examples]

[['announced',
  'biggest',
  'headlining',
  'show',
  'date',
  'december',
  '13th',
  'concert',
  '20',
  '000',
  'capacity',
  'london',
  'o2',
  'arena'],
 ['people', 'never', 'talk', 'handing', 'paycheck'],
 ['posted', 'june', '20', '2014', 'stream', 'tagged', 'outsiders', 'polydor'],
 ['cop', 'free', 'download', 'one', 'via', 'bearson', 'soundcloud', 'page'],
 ['developed',
  'protective',
  'feeling',
  'portrayed',
  'time',
  'im',
  'acutely',
  'aware',
  'issues',
  'face',
  'root',
  'causes',
  'issues']]

In [15]:
from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary(texts)

In [16]:
from gensim.models.ldamodel import LdaModel

n_topics = 5
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

corpus = [dictionary.doc2bow(text) for text in texts]
# Train the model on the corpus.
#lda = LdaModel(corpus, id2word=dictionary, num_topics=n_topics, iterations=iterations, passes=passes, eval_every=eval_every)
lda = LdaModel(corpus, id2word=dictionary, num_topics=n_topics)
lda.print_topics(n_topics)

[(0,
  '0.030*"new" + 0.022*"single" + 0.018*"album" + 0.017*"track" + 0.013*"first" + 0.013*"ep" + 0.013*"release" + 0.012*"debut" + 0.012*"year" + 0.010*"released"'),
 (1,
  '0.012*"one" + 0.011*"music" + 0.009*"like" + 0.009*"get" + 0.009*"time" + 0.007*"song" + 0.007*"know" + 0.007*"us" + 0.006*"sure" + 0.006*"listen"'),
 (2,
  '0.009*"pop" + 0.006*"rock" + 0.005*"vocals" + 0.005*"guitar" + 0.004*"dance" + 0.004*"song" + 0.004*"track" + 0.004*"music" + 0.004*"indie" + 0.004*"like"'),
 (3,
  '0.008*"music" + 0.008*"via" + 0.008*"tour" + 0.006*"records" + 0.006*"11" + 0.006*"10" + 0.006*"festival" + 0.005*"london" + 0.005*"october" + 0.005*"club"'),
 (4,
  '0.015*"track" + 0.011*"like" + 0.010*"vocals" + 0.009*"song" + 0.008*"pop" + 0.007*"sound" + 0.005*"one" + 0.005*"music" + 0.005*"house" + 0.005*"production"')]

In [17]:
from operator import itemgetter

for i in range(n_examples):
    print(max(lda[corpus[i]],key=itemgetter(1)), ds.normalize_string(sents[i]))

(3, 0.9463464) artist just announced their biggest headlining show to date a december 13th concert at the 20 000 capacity london o2 arena
(1, 0.6936159) and then there are the people youd never talk to if they werent handing you a paycheck
(3, 0.37547305) posted on june 20 2014 in stream and tagged artist outsiders polydor
(1, 0.9104812) cop a free download of that one via bearson s soundcloud page
(2, 0.60289574) i ve developed this protective feeling about how were portrayed and at the same time im acutely aware of the issues we face and the root causes of these issues


In [18]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.176788 -0.024853       1        1  27.155352
0      0.010818  0.182707       2        1  25.492100
1     -0.078168  0.159743       3        1  20.995756
2     -0.112820 -0.254344       4        1  13.350050
3      0.356958 -0.063253       5        1  13.006740, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
482   Default   7388.000000     single   7388.000000  30.0000  30.0000
210   Default  12815.000000        new  12815.000000  29.0000  29.0000
193   Default   6942.000000      album   6942.000000  28.0000  28.0000
375   Default   4142.000000      debut   4142.000000  27.0000  27.0000
75    Default   4500.000000         ep   4500.000000  26.0000  26.0000
238   Default   4050.000000       year   4050.000000  25.0000  25.0000
301   Default   3438.000000   released   3438.000000  24.0000  24.0000
380   Default   4609.000000    release   4609.000000  23.0000  23.0000
111   Default   5114.000000      first   5114.000000  22.0000  22.0000
317   Default   4502.000000     vocals   4502.000000  21.0000  21.0000
421   Default   2956.000000       last   2956.000000  20.0000  20.0000
638   Default   2858.000000   producer   2858.000000  19.0000  19.0000
436   Default   2052.000000       know   2052.000000  18.0000  18.0000
82    Default  13310.000000      track  13310.000000  17.0000  17.0000
221   Default   2381.000000      based   2381.000000  16.0000  16.0000
538   Default   1844.000000       tour   1844.000000  15.0000  15.0000
264   Default   5495.000000        pop   5495.000000  14.0000  14.0000
1043  Default   2078.000000      since   2078.000000  13.0000  13.0000
149   Default   3188.000000        get   3188.000000  12.0000  12.0000
33    Default   2422.000000        via   2422.000000  11.0000  11.0000
415   Default   1662.000000       sure   1662.000000  10.0000  10.0000
125   Default   7311.000000       like   7311.000000   9.0000   9.0000
321   Default   1541.000000      check   1541.000000   8.0000   8.0000
40    Default   1903.000000         im   1903.000000   7.0000   7.0000
564   Default   1644.000000    records   1644.000000   6.0000   6.0000
2141  Default   1026.000000         11   1026.000000   5.0000   5.0000
71    Default   2289.000000      years   2289.000000   4.0000   4.0000
304   Default   1625.000000     called   1625.000000   3.0000   3.0000
30    Default   7532.000000        one   7532.000000   2.0000   2.0000
64    Default    969.000000         10    969.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
2390   Topic5    956.675232   festival    968.663147   2.0272  -5.1783
171    Topic5    359.803314         23    361.036407   2.0363  -6.1562
2386   Topic5    359.567535         21    361.331665   2.0348  -6.1568
1213   Topic5    640.816284      april    648.444397   2.0279  -5.5790
2148   Topic5    412.041901          4    416.259766   2.0295  -6.0206
146    Topic5    535.660950          5    545.770874   2.0210  -5.7582
1709   Topic5    640.116638          6    664.807739   2.0019  -5.5801
599    Topic5    654.047729          3    682.148865   1.9976  -5.5585
2404   Topic5    414.552246         19    427.321777   2.0094  -6.0145
2759   Topic5    594.140259   november    652.581238   1.9459  -5.6546
561    Topic5    708.555664         12    807.916016   1.9085  -5.4785
1402   Topic5    874.753540    october   1042.568115   1.8642  -5.2678
79     Topic5    611.792725      march    699.556763   1.9056  -5.6253
538    Topic5   1283.870728       tour   1844.258545   1.6775  -4.8841
1480   Topic5    806.759888         uk   1106.281738   1.7240  -5.3487
584    Topic5    843.003906       club   1201.307373   1.6855  -5.3047
564    Topic5   1046.157227    records   1644.808228   1.5872  -5.0888
33     Topic5   1339.991211  

In [19]:
from collections import Counter
Counter([max(lda[corpus[i]], key=itemgetter(1))[0] + 1 for i in range(len(texts))])

Counter({4: 7872, 2: 25917, 3: 8538, 1: 28809, 5: 33331})

In [20]:
out = sorted([f'{max(lda[corpus[i]], key=itemgetter(1))[0] + 1} {ds.normalize_string(sents[i])}\n' for i in range(len(texts))])
with open('pairs_sentence_topics.txt', 'w') as f:
    f.writelines(out)

^using this file to select desirable/preferred topics.

In [21]:
def get_example(i, pairs, input_side, output_side, random_state, device):
    pair = pairs[i]

    inp = model.word_tensor(input_side, pair[0]).to(device)
    target = model.word_tensor(output_side, pair[1]).to(device)
    condition = torch.tensor(pair[2], dtype=torch.float).unsqueeze(0).to(device) if len(pair) == 3 else None

    return inp, target, condition

In [22]:
import time
from pytorchtextvae import model

fav_topic = 3

labels = np.zeros(len(texts), dtype=int)
embeds = []
start = time.time()

# debug vars
ts = []
iss = []

for i in range(n_samples):
    pair_i = random_state.choice(len(pairs))
    iss.append(pair_i)
    ts.append(pairs[pair_i][0])
    input, target, condition = get_example(pair_i, pairs, input_side, output_side, random_state, DEVICE)
    with torch.no_grad():
        _, _, z, _ = vae(input, target, condition, DEVICE, temp)
        squashed_condition = vae.decoder.c2h(condition)
        decode_embed = torch.cat([z, squashed_condition], 1)
        embeds.append(decode_embed)
    
    topic_i = max(lda[corpus[pair_i]],key=itemgetter(1))[0] + 1
    labels[pair_i] = 1 if topic_i == fav_topic else -1

print(f'runtime: {time.time() - start:.2f}s')
list(zip([labels[j] for j in iss], ts[:n_samples]))

runtime: 113.99s


[(-1, 'it reminds me i m foreign in a place far away'),
 (-1,
  'you can find it on in your prime a new ep from salvat dropping next month'),
 (-1,
  'undeterred the pair reevaluated and entered the music business via a different avenue collaborating with the likes of maggie rogers and charles fauna'),
 (-1,
  'long story short itll get under your skin in no time so well done everyone'),
 (-1, 'new zealands caleb and georgia nott better known as broods revealed'),
 (-1,
  'but its not all spot the reference or derivative pop predicated on its predecessors'),
 (-1,
  'the soviet union had exhausted and old values were anathematized because of reconstruction'),
 (-1, 'and i love the folk elements and tender lyrics he crafts'),
 (-1,
  'sitename november 26 2014 following on from the grand unveiling of its title track and pelican pioneering producer bonobo has now revealed a third cut from his inbound flashlight ep which drops december 2nd via ninja tune'),
 (-1,
  'bicep are setting off 

## Data

In [23]:
batch_size = 16
embed_size = embeds[0].size(1)

In [24]:
from fastai.dataset import *

class LatentDataset(Dataset):
    def __init__(self, embeds, labels): self.embeds,self.labels = embeds,labels
    def __getitem__(self, idx): return A(self.embeds[idx], self.labels[idx])
    def __len__(self): return len(self.embeds)
    
ds = LatentDataset(embeds, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [25]:
md.trn_ds[0], md.trn_ds[0][0].shape

([array([[ 0.93375, -1.01354,  1.95816, -0.43546,  0.77704,  0.49258,  2.27868, -0.19384, -0.9598 ,  0.16328,
          -0.71583, -2.76212,  2.43436, -1.52841,  0.42632, -0.78335,  1.86925,  1.85429,  0.58314,  1.1069 ,
          -1.45151, -2.89545, -1.9075 , -1.16109,  2.12313, -1.24098,  3.92334, -1.06145,  0.56524, -0.48142,
           0.76074,  1.14141,  1.85901,  1.49063, -3.79368, -0.62984,  0.46462,  1.33705,  2.15374,  1.07177,
          -0.24337, -0.46972,  0.89422, -0.76085, -0.51082,  0.2632 ,  0.40455,  0.20931, -0.57702, -0.83613,
           0.51499,  1.23155, -2.26053, -1.01639,  1.69184, -1.36231,  0.20127,  0.94662,  2.21778,  1.17812,
          -1.94448, -2.29459,  0.22887,  1.00351,  1.33151, -0.93714, -0.59258, -2.78561, -0.18092,  2.01536,
           1.04038,  0.59318, -3.357  ,  0.74279, -0.78432, -1.24676,  0.36504,  1.27641,  2.23924, -0.56912,
           1.15502,  3.64338,  1.09483,  1.53716,  1.11848,  0.43782, -2.46393,  1.59396,  1.89401, -0.53894,
          

## Model

In [26]:
n_hidden = 1024
lr = 3e-4
fixed_genres = torch.FloatTensor(dataset.encode_genres(['edm', 'tropical house'])).to(DEVICE)

In [27]:
import torch.optim as optim
import torch.nn as nn

class LCGAN_D(nn.Module):
    '''Discriminator'''
    def __init__(self, n_embed, n_hidden=n_hidden, n_output=1):
        super(LCGAN_D, self).__init__()
        
        self.i2h = nn.Linear(n_embed, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        self.h2o = nn.Linear(n_hidden, n_output)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, emb):
        x = emb
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        v = self.sigmoid(self.h2o(x))
        
        return v

class LCGAN_G(nn.Module):
    '''Generator'''
    def __init__(self, n_embed, n_hidden=n_hidden):
        super(LCGAN_G, self).__init__()
        self.n_embed = n_embed
        
        self.i2h = nn.Linear(n_embed, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        # hidden-to-gating mechanism
        self.h2g = nn.Linear(n_hidden, 2*n_embed)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, emb):
        x = emb
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        x = self.h2g(x)
        
        # gating mechanism: allow network to remember/forget
        # what it wants to about the original emb(edding) and x
        emb_mid = x[:, self.n_embed:]
        gates = self.sigmoid(x[:, :self.n_embed])
        demb = gates * emb_mid # TODO: why naming?
        emb_prime = (1 - gates)*emb + demb
        
        return emb_prime

In [28]:
class LCGAN(nn.Module):
    def __init__(self, D, G, batch_size=batch_size):
        super(LCGAN, self).__init__()
        self.batch_size = batch_size
        
        self.D = D
        self.G = G

    def train(self):
        self.D.train()
        self.G.train()
        
    def eval(self):
        self.D.eval()
        self.G.eval()
        
    def forward(self, emb=None):       
        if emb is not None:
            # train discriminator
            #embed = to_embed(z, fixed_genres)
            v = self.D(emb)
            return v
        else: # train GAN
            # gaussian random noise
            emb_prior = torch.randn(self.batch_size, self.G.n_embed).to(DEVICE)
            
            emb_prime = self.G(emb_prior)
            v_prime = self.D(emb_prime)
            
            return v_prime

## Training

In [29]:
gan = LCGAN(LCGAN_D(embed_size).to(DEVICE), LCGAN_G(embed_size).to(DEVICE)).to(DEVICE)

In [30]:
import fastai

fastai.core.set_trainable(gan.D, True)
fastai.core.set_trainable(gan.G, True)

opt_d = optim.Adam(gan.D.parameters(), lr=lr)
opt_g = optim.Adam(gan.G.parameters(), lr=lr)

In [31]:
# test what the GAN is doing before any training
for i in range(10):
    print(generate(fixed_genres, gan)[0])

at it with a modern disco sound that you it in a chilled look in image courtesy of disco world
the UNK is something that come out of a records to sing along from the artist and an interview with UNK march
the brooklyn canadian trio artist has came up with a part of UNK and keep an eye on UNK march
the it will be signed to UNK yet but with some of the UNK to as an eye on record apart
i have UNK step away from the image courtesy of artist and and better known as they keep an eye over the show
it took over the UNK and yet another song that reminds me in the case for me ahead
with one of the UNK UNK UNK just with some of the album as ive been impact on the these days
UNK below along with the the version duo artist is back in a UNK and being featured for this live performance
it might be one of the impressed us with his sultry vocals it has some sort an of live performance
we present in his sound so many of music with a more and and in which to his ethereal atmosphere


In [32]:
#from tqdm import tqdm

# adapted from: https://github.com/fastai/fastai/blob/master/courses/dl2/wgan.ipynb
def train(n_iter, alternate=False, first=False):
    gen_iters = 0
    for epoch in trange(n_iter):
        gan.train()
        data_iter = iter(md.trn_dl)
        i, n = 0, len(md.trn_dl)
        
        def train_G():
            ''' Train generator '''
            nonlocal gen_iters
            
            fastai.core.set_trainable(gan.D, False)
            fastai.core.set_trainable(gan.G, True)

            gan.G.zero_grad()

            #print(i, n)
            loss_g = - torch.log(gan()).mean()
            loss_g.backward()
            opt_g.step()
            gen_iters += 1
            
            return loss_g

        def train_D():
            ''' Train discriminator '''
            nonlocal i
            
            fastai.core.set_trainable(gan.D, True)
            fastai.core.set_trainable(gan.G, False)
            d_iters = 100 if (first and (gen_iters < 25) or (gen_iters % 500 == 0)) else 3
            j = 0

            while (j < d_iters) and (i < n):
                j += 1; i += 1
                batch = next(data_iter)
                #print(j, i, batch[0].size(), batch[1].size())
                emb_real = batch[0].to(DEVICE)
                v = gan(emb_real).to(DEVICE)

                gan.D.zero_grad()

                #loss_d = - (batch[1] * torch.log(v) + (1.0-batch[1]) * torch.log(1.0 - v)).mean()
                loss_d = - (batch[1].to(DEVICE) * torch.log(v) + (1.0-batch[1].to(DEVICE)) * torch.log(1.0 - v)).mean()
                loss_d.backward()
                opt_d.step()
                
                pbar.update()
                
            return loss_d
        
        with tqdm(total=n) as pbar:
            while i < n:
                if alternate:
                    # train discriminator
                    loss_d = train_D()
                    # then train generator a little bit
                    loss_g = train_G()
                else:
                    # train generator only
                    i += 1
                    loss_g = train_G()
                    pbar.update()
        
        if alternate:
            print(f'Loss_D {to_np(loss_d)}; Loss_G {to_np(loss_g)}; ')
        else:
            print(f'Loss_G {to_np(loss_g)}; ')

In [33]:
def train_and_generate(gan, n_epoch, genres=fixed_genres, alternate=False, n_sample=10):
    train(n_epoch, alternate)
    res = []
    for i in range(n_sample):
        res.append(generate(genres, gan))
    return res

In [34]:
[res[0] for res in train_and_generate(gan, 1, alternate=True)]

100%|██████████| 7/7 [00:00<00:00, 18.47it/s]
Loss_D 0.012704020366072655; Loss_G 2.137624979019165; 
100%|██████████| 1/1 [00:00<00:00,  2.60it/s]

['UNK is the sort of being trying to come to earlier this year and turning it down some final yourself',
 'artist has a fan of new which is in the complete with soulful vocals and UNK has become one of these tracks',
 'more UNK and guest vocals from swedish aka UNK to no one of whats going on being ahead',
 'one of of bristol based producer artist has went UNK UNK UNK UNK and the sort of vocal work',
 'new track just fell in returns with a follow up to UNK and turning dive UNK a spot on vocal performance',
 'along with a called UNK which with back in the UNK and knows how to keep your down',
 'UNK is the version of artist UNK to right here and killing it with the definitely if you enjoy',
 'according to the music that UNK UNK UNK UNK and is one of the music that i had to dive into these work',
 'that continues has came up with a the label in different production which is set to fit for the vocal set',
 'if to get away from the because of whats going on vocals and feel good fit of these

In [35]:
[res[0] for res in train_and_generate(gan, 1)]

100%|██████████| 7/7 [00:00<00:00, 45.50it/s]
Loss_G 1.5952479839324951; 
100%|██████████| 1/1 [00:00<00:00,  6.40it/s]

['UNK UNK UNK is the three piece of heart now and with a full of artists such as ep works',
 'UNK UNK UNK UNK UNK a piece of love is filled with soulful vocals and and aside from your own',
 'he impressed returns with a piece of track that is taken over four years ago but in the piece of live season',
 'to UNK UNK UNK UNK UNK is a bit of stripped down and los angeles of his music video label',
 'UNK is looking for the electro pop song that reminds us into a full of warm and combined and an ethereal atmosphere',
 'is is as part of UNK UNK UNK to has impressed us with a slew of your earlier today boy path',
 'UNK is known for a song that that impressed us with a waves of beats and in addition to the new atmosphere',
 'the UNK UNK UNK is a full of guitar beats and electro but with a hint of live instrumentation',
 'its a slice of UNK that has impressed us with a warm and unique and as part of my instrumentation',
 'artist is back UNK UNK UNK UNK UNK that has impressed us with another bran

In [36]:
[res[0] for res in train_and_generate(gan, 1)]

100%|██████████| 7/7 [00:00<00:00, 54.47it/s]
Loss_G 1.2359094619750977; 
100%|██████████| 1/1 [00:00<00:00,  7.49it/s]

['electro pop trio artist has impressed us for a bit UNK UNK UNK UNK to keep an eye our live again',
 'the UNK UNK has with a piece of rise to and us with the image courtesy of oliver atmosphere',
 'courtesy of UNK UNK really impressed us with a of UNK and look at the courtesy of dj list',
 'the UNK has a bit of all over the past year but with his hint on ethereal atmosphere',
 'with the UNK which with just as part for a UNK UNK continues to showcasing his vocal again',
 'UNK is moving away from the likes of UNK and UNK who features a sort of all over an ethereal atmosphere',
 'one of the best known for over UNK years but im but with the image courtesy by pop radio',
 'artist artist has released as part of the UNK and with a UNK which music hailing from came together',
 'from the UNK UNK UNK UNK is a sort of UNK below us with the hint of major label',
 'the sort the trio artist has impressed back with a sort of warm and best known of of ethereal atmosphere']

In [37]:
[res[0] for res in train_and_generate(gan, 1)]

100%|██████████| 7/7 [00:00<00:00, 63.69it/s]
Loss_G 0.988798975944519; 
100%|██████████| 1/1 [00:00<00:00,  8.80it/s]

['brand new UNK who grew up in the kind of vocals and serve as the sort the live afternoon',
 'the electro pop of artist has come back in with UNK UNK and happens to be featured on the video for music afternoon',
 'artist from the UNK in UNK and with a piece of track which is totally taken on the live afternoon',
 'is the sort of UNK who grew up in the UNK and and bound to being described this wonderful afternoon',
 'one of of UNK UNK artist is combined in the signature sound and serve as some of its ethereal afternoon',
 'UNK we are took over the UNK and some and his voice as well as a working on new afternoon',
 'back in obsessed with the songs that come millions of UNK and a better known on their previous afternoon',
 'in the image courtesy of UNK who has impressed us with a UNK be featured on the live afternoon',
 'artist has teamed up with the UNK UNK and what happens to with a handful of UNK be featured on of live afternoon',
 'UNK has teamed up with UNK UNK UNK and happens to be

## Inference with saved models

In [47]:
def save():
    save_state_filename = 'ganG_state.pt'
    torch.save(gan.G.state_dict(), save_state_filename)
    print('Saved as %s' % (save_state_filename))
    
save()

Saved as ganG_state.pt


In [55]:
def gan_generate(vae, condition, n_latent, ganG, max_sample=False, trunc_sample=True):
    with torch.no_grad():
        ganG.eval()
        z = torch.randn(1, n_latent).to(DEVICE)
        decode_embed = to_embed(z, condition).to(DEVICE)
        z_prime = ganG(decode_embed)

        generated = vae.decoder.generate_with_embed(z_prime, 50, temp, DEVICE, max_sample=max_sample, trunc_sample=trunc_sample)
        generated_str = model.float_word_tensor_to_string(output_side, generated)

        EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

        if generated_str.endswith(EOS_str):
            generated_str = generated_str[:-5]

        # flip it back
        return generated_str[::-1], z, z_prime

In [56]:
ganG = LCGAN_G(embed_size).to(DEVICE)
ganG.load_state_dict(torch.load('ganG_state.pt'))

gan_generate(vae, torch.FloatTensor(dataset.encode_genres(['hip hop','pop','pop rap','rap','trap music'])).to(DEVICE), n_latent, ganG)

('a form of UNK and UNK is no wonder throughout a track just interested welcome addition to their music career',
 tensor([[ 0.3426, -0.4434,  0.6537, -1.0167,  0.2749,  0.3956, -1.0323,
           1.9262, -0.2886,  0.0875, -1.1526,  0.5510, -0.6338, -0.5600,
          -1.1005, -0.0089, -0.0115,  0.3233, -1.4833, -0.4620,  0.2851,
          -0.1228, -2.3361, -1.1106,  0.3530,  0.6186, -0.2250, -1.9346,
           0.4420, -1.3540, -0.9850,  1.5157,  1.8120,  0.8877, -1.1195,
           0.1646,  0.2872, -1.1713, -0.0716, -0.2902, -0.3561,  0.2219,
           0.8907, -1.1704, -0.0048, -0.6173,  1.3518,  0.4813, -0.1438,
          -1.4539,  0.2248, -0.2255,  1.8956,  0.4242,  0.5299, -0.6540,
           0.7414, -0.7336, -0.3256,  1.0842,  0.2970, -1.3575, -1.0421,
          -1.7367, -0.6553,  0.8228,  1.5621, -0.5879, -0.6136, -0.1467,
          -0.0047,  1.4218,  1.6615,  0.0985, -0.0368, -0.6737,  0.0706,
          -1.5198, -0.6521, -1.7595,  0.6300,  2.0502,  1.0153, -0.0525,
           

## Extras

retrain discriminator with new samples... `z_prime`s that the discriminator is still not rejecting strongly enough

In [36]:
labels = np.ones(len(g2), dtype=int)
gens_lose = list(set([i for b in banned for i in np.where([b in res[0].split() for res in g2])[0]]))
#gens_keep = list(set(range(len(g2))) - gens_lose)
labels[gens_lose] = -1
zs_keep = np.array([res[2] for res in g2], dtype=object)

labels        

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [32]:
ds = LatentDataset(zs_keep, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [33]:
[res[0] for res in train_and_generate(gan, 1, alternate=True)]

100%|██████████| 7/7 [00:00<00:00, 284.11it/s]
Loss_D 1.193210244178772; Loss_G 1.1896806955337524; 


['artist has teams up with a a one of the UNK and it would be taken from i don no feat',
 'this with a remix of UNK and trying to take of the UNK theres no feat',
 'the remix of UNK has been trying to take of the UNK theres no feat',
 'UNK is a listen to the UNK and coming side of what to look at feat',
 'a new track called UNK and serves as one of the trying would be ready for at times',
 'artist has up with a UNK and one of the track will be inspired as no feat',
 'artist is back in the likes of UNK and serves as a no times',
 'artist who reminds back to a more stuck on the UNK and theres many feat',
 'he continues to serves as one of the week what i look no feat',
 'with a layers of UNK and and UNK of the perfect trying i look at feat']